In [4]:
from google.colab import drive
drive.mount('/content/drive')

! pip install mido
! pip install music21
! pip install np_utils
! pip install pygame
! pip install keras_self_attention

from mido import MidiFile, merge_tracks, tempo2bpm
import mido
from music21 import *
import music21
import keras
from keras import optimizers
from keras.preprocessing import sequence
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Bidirectional, Dropout, GlobalMaxPooling1D, Activation
from keras_self_attention import SeqSelfAttention
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import glob
import np_utils
from collections import Counter
import pandas as pd
from datetime import datetime
import matplotlib.patches as patches
import matplotlib.pyplot as plt


%matplotlib inline

Mounted at /content/drive
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 11.5MB 250kB/s 
  Created wheel for keras-self-attention: filename=keras_self_attention-0.47.0-cp36-none-any.whl size=17288 sha256=f4dc12c6c706a0bac86dde8d19c2fd534386195de0987d594d8453735a6bd8d5
  Stored in directory: /root/.cache/pip/wheels/70/87/01/76c703d5401b65e323927c1fdc665f3fb143282ff67d71e859
Successfully built keras-self-attention


In [30]:
DIRECTORY = './'
# DIRECTORY = './drive/My Drive/'
DATASET_DIR = DIRECTORY + 'doug_mckenzie_midi/'
CHORDS_DATAFRAME_NAME = 'ChordsData.csv'
NOTES_DATAFRAME_NAME = 'NotesData.csv'
CHORDS_WEIGHTS_NAME = 'Lstm_Chord_Weights.hdf5'
NOTES_WEIGHTS_NAME = 'Lstm_Melody_Weights.hdf5'
SEED_SONG_NAME = 'SmallDay'
GENERATED_CHORDS_FILE = 'GeneratedChords.mid'
GENERATED_NOTES_FILE = 'GeneratedNotes.mid'

# DATA PREPROCESSING INTO DATAFRAME

In [2]:
threshold = 25

In [9]:
def getMidiFileMeta(midiFile):
    bpmSum = 0
    bpmCount = 0;
    bpm = None
    timeSignature = None
    key = None
    for message in merge_tracks(midiFile.tracks):
        if message.type == 'set_tempo':
            bpmSum += tempo2bpm(message.tempo)
            bpmCount += 1
        if message.type == 'time_signature':
            timeSignature=f"{message.numerator}/{message.denominator}"
        if message.type == 'key_signature':
            key=message.key

    if bpmCount != 0:
        bpm = bpmSum/bpmCount
    return bpm, timeSignature, key
  
def parseSplitChordsAndNotes(threshold, midiFile, midiTrack, noteLimit=99999):
    notes = []
    allNotes = []
    chords = []
    currentChord = []
    noteMemo = {}
    thisChordStartTime = None
    lastChordStartTime = None
    lastNoteStartTime = None
    lastChord = None
    time = 0
    i = 0 
    
    bpm, timeSignature, key = getMidiFileMeta(midiFile)
    for message in midiTrack:
        time += message.time
        noteOn = (message.type == 'note_on' and message.velocity > 0)
        noteOff = (message.type == 'note_on' and message.velocity == 0)
        
        if noteOn:
            i += 1
            if i > noteLimit:
                break;
            if noteMemo.get(message.note) == None:
              note = {
                  # "message": message,
                  "midi": message.note,
                  "velocity": message.velocity,
                  "time": time,
                  "relativeTime": message.time,
                  "duration": 0
              } 

              allNotes.append(note)
              
              noteMemo[message.note] = note

              #initialize this
              if thisChordStartTime == None:
                thisChordStartTime = time
              
              if lastChordStartTime == None:
                lastChordStartTime = time

              #=== IF INCOMING NOTE EXCEEDS WINDOW: START NEW CHORD
              timeDiff = time-thisChordStartTime
              if timeDiff > threshold:
                #=== HANDLE OLD CHORD                  
                numberOfNotes = len(currentChord)
                #=== If it is a chord
                if numberOfNotes > 1:
                    #=== Take reference to timestamp of previous chord
                    timeSinceLastChord = thisChordStartTime - lastChordStartTime
                    chord = {
                      "relativeTime": timeSinceLastChord,
                      "time": thisChordStartTime,
                      "bpm": bpm,
                      "timeSignature": timeSignature,
                      "notes": currentChord
                    } 

                    chords.append(chord)
                    lastChord = chord
                    
                    #=== Timestamp this chord start for next chord reference
                    lastChordStartTime = thisChordStartTime                    
                #=== If it is a note
                else:                
                    noteDict = currentChord[0]
                    noteDict["chord"] = lastChord;
                    noteDict["relativeTime"] = thisChordStartTime-lastNoteStartTime if lastNoteStartTime != None else 0 #we want to manually calc the relativeTime based on objective time
                    notes.append(noteDict)

                    #this note is only pushed when a new distant note is found
                    #therefore the start time of note being pushed is the this chord start time
                    lastNoteStartTime = thisChordStartTime 
                
                #=== START OF NEW CHORD: BEGINS
                thisChordStartTime = time #set as none so that next note that is picked up begins a new chord
                currentChord = []   
              
              #append chord both ways
              currentChord.append(note)
                  

        if noteOff:
            memoNote = noteMemo.get(message.note)
            if memoNote != None:
                memoNote["duration"] = (time - memoNote["time"]) 
                noteMemo[message.note] = None

    return chords, notes, allNotes

class NoteData: 
  def __init__(self, parsedNote):
    self.midiNote = parsedNote["midi"]
    self.velocity = parsedNote["velocity"]
    self.duration = parsedNote["duration"]
    self.relativeTime = parsedNote["relativeTime"]
    self.chord = ChordData(parsedNote["chord"]) if parsedNote["chord"] != None else None

  def normalizeToC(self, key):
    self.chord.normalizeToC(key)
    self.midiNote -= key.tonic.pitchClass
  
  def features(self):    

    noteDict = {
        "midi": self.midiNote,
        "velocity": self.velocity,
        "duration": self.duration,
        "relativeTime": self.relativeTime,        
    }

    if self.chord:
      chordFeatures = self.chord.features()
      noteDict["chordPitch"] =  chordFeatures["root"]
      noteDict["chordForte"] = chordFeatures["forte"]
      noteDict["chordQuality"] = chordFeatures["quality"]
    else:
      print("note has no chord")
      noteDict["chordPitch"] = None
      noteDict["chordForte"] = None
      noteDict["chordQuality"] = None
    
    return noteDict

    
      
  def __str__(self):    
    return f"""
    midiNote:{self.midiNote} 
    duration: {self.duration}
    velocity: {self.velocity}
    time since last chord: {self.relativeTime}    
    """
  
  @staticmethod
  def m21NotesFromParsedNotes(parsedNotes):
    return [music21.note.Note(parsedNote["midi"]) for parsedNote in parsedNotes]


class ChordData:
  def __init__(self, parsedChord):
    self.parsedNotes = parsedChord["notes"]
    self.duration = np.max([note["duration"] for note in self.parsedNotes])
    self.velocity = np.mean([note["velocity"] for note in self.parsedNotes])
    self.relativeTime = parsedChord["relativeTime"]
    self.bpm = parsedChord["bpm"]

    self.m21Notes = NoteData.m21NotesFromParsedNotes(self.parsedNotes)
    self.m21Chord = music21.chord.Chord(self.m21Notes)
    self.keyPitchClass = None
    

  def normalizeToC(self, key):
    # print(f"transposed from key: {key.tonic} chord: {self.m21Chord.root()}" )
    self.m21Chord = self.m21Chord.transpose(-key.tonic.pitchClass) #normalise to key
    # print(f"transposed to chord: {self.m21Chord.root()}" )

  def features(self):
    return {
        "root": self.m21Chord.root().pitchClass,
        "velocity": self.velocity,
        "relativeTime": self.relativeTime,
        "duration": self.duration,
        "forte-cardinality": self.m21Chord.pitchClassCardinality,
        "forte-class": self.m21Chord.forteClassNumber,
        "forte": self.m21Chord.forteClass,
        "bpm": self.bpm,
        "quality": self.m21Chord.quality,
        "duration": self.duration
    }

  def __str__(self):    
    return f"""
    root:{self.m21Chord.root()} 
    duration: {self.duration}
    velocity: {self.velocity}
    time since last chord: {self.relativeTime}
    root pitch class:{self.m21Chord.root().pitchClass} 
    quality:{self.m21Chord.quality} 
    no of notes:{len(self.parsedNotes)}
    common name: {self.m21Chord.commonName}
    pitched common name: {self.m21Chord.pitchedCommonName}
    forte class w inversion: {self.m21Chord.forteClassTn}
    forte class w/o inversion: {self.m21Chord.forteClassTnI}
    bpm: {self.bpm}
    """

def keyFromChordsData(inpChordsData):   
  m21Stream = music21.stream.Stream([chord.m21Chord for chord in inpChordsData])
  key = m21Stream.analyze('key')
  normalisedToMajorKey = key.asKey(mode='major') #normalise all keys to major keys
  return normalisedToMajorKey

def normalizeChordsDataKey(chordsData):
  key = keyFromChordsData(chordsData)
  for chord in chordsData:
    chord.normalizeToC(key)

def normalizeNotesData(notesData, key):
  for noteData in notesData:
    noteData.normalizeToC(key)

chordInputData = []
noteInputData = []

for temp in glob.glob(DATASET_DIR + "*.mid"): 
  try:
    print("parsing {}".format(temp))
    inputFile = MidiFile(temp)  

    trackIdx = None
    maxMessageCount = 0
    for i, track in enumerate(inputFile.tracks):
      messageCount = len(track)
      if messageCount > maxMessageCount:
        trackIdx = i
        maxMessageCount = messageCount
    
    #exceptions    
    if temp == DATASET_DIR + 'PeanutVendor.mid' or temp == DATASET_DIR + 'Spain4.mid':
      trackIdx = 1

    track = inputFile.tracks[trackIdx]
    chords, notes, allNotes = parseSplitChordsAndNotes(threshold, inputFile, track)
    chordsData = [ChordData(chord) for chord in chords]
    normalizeChordsDataKey(chordsData) #normalize key to C
    for chordData in chordsData:
      chordInputData.append(chordData.features())

    #notes
    notesData = [NoteData(note) for note in notes]
    for noteData in notesData:
      noteInputData.append(noteData.features())
  except Exception as error:
    # raise error
    print(error)
    # break

chordInputData = np.array(chordInputData)
noteInputData = np.array(noteInputData)

#INTO DATAFRAME
chordsDataframe = pd.DataFrame(data=[chordDict.values() for chordDict in chordInputData], columns=chordInputData[0].keys())
chordsDataframe.to_csv(DIRECTORY + CHORDS_DATAFRAME_NAME)

notesDataframe = pd.DataFrame(data=[noteDict.values() for noteDict in noteInputData], columns=noteInputData[0].keys())
notesDataframe.to_csv(DIRECTORY + NOTES_DATAFRAME_NAME)
notesDataframe


parsing ./drive/My Drive/doug_mckenzie_midi/ABeautifulFriendship.mid
parsing ./drive/My Drive/doug_mckenzie_midi/AFineRomance.mid
parsing ./drive/My Drive/doug_mckenzie_midi/AChildIsBorn2.mid
parsing ./drive/My Drive/doug_mckenzie_midi/Aghostofachance.mid
note has no chord
note has no chord
note has no chord
parsing ./drive/My Drive/doug_mckenzie_midi/AHouseisNot.mid
note has no chord
parsing ./drive/My Drive/doug_mckenzie_midi/ASleepinBee.mid
note has no chord
parsing ./drive/My Drive/doug_mckenzie_midi/ARemark.mid
parsing ./drive/My Drive/doug_mckenzie_midi/AlmostBlue.mid
note has no chord
parsing ./drive/My Drive/doug_mckenzie_midi/AllTheThingsGroup.mid
parsing ./drive/My Drive/doug_mckenzie_midi/AllTheThingssoloandtrio.mid
parsing ./drive/My Drive/doug_mckenzie_midi/AngelEyes.mid
parsing ./drive/My Drive/doug_mckenzie_midi/Answer%20me%20My%20LoveCorrected2.mid
parsing ./drive/My Drive/doug_mckenzie_midi/AnAffairToRemember.mid
note has no chord
note has no chord
note has no chord
no

,midi,velocity,duration,relativeTime,chordPitch,chordForte,chordQuality
0,76,83,17,0,9.0,4-26,minor
1,75,76,18,28,9.0,4-26,minor
2,72,88,16,430,7.0,3-9,other
3,74,115,21,44,7.0,3-9,other
4,77,73,113,76,4.0,3-8B,other
...,...,...,...,...,...,...,...
83239,52,27,799,136,11.0,3-7A,minor
83240,43,44,1776,699,11.0,3-7A,minor
83241,64,62,387,26,11.0,3-7A,minor
83242,42,56,427,2522,0.0,4-20,major


### Visualise Chord Melody Split

In [ ]:
midiFile = MidiFile(DATASET_DIR + 'AChildIsBorn2.mid')

for track in midiFile.tracks:
  print(len(track))

for track in midiFile.tracks:
  count = 0
  for message in track:
    if message.type == 'note_on':
      count += 1


  print(count)
chords, notes, allNotes = parseSplitChordsAndNotes(threshold, midiFile, merge_tracks(midiFile.tracks), 100)

GRAPH_X_AXIS = 5000
def drawChords(inp, threshold, rows, columns, i):    
  fig = plt.figure()
  fig = fig.add_subplot(111)
  fig.title.set_text('With threshold: {0}'.format(threshold))
  plt.ylim((100, 270))
  plt.xlim((0, 6000))
  plt.xlabel('time')
  plt.ylabel('pitch')
  
  colors = ['red', 'green', 'blue', 'yellow']
  mux = 0
  toggle = True #used to alternate chord colors for visibility
  for chord in inp:
      for note in chord["notes"]:
          width = note["duration"]
          height = 2
          x = note["time"]
          y = note["midi"]*(height+1)
          fig.add_patch(patches.Rectangle((x, y), width, height, color=colors[mux] ))
      mux = (mux + 1) % len(colors)
  return fig

def drawNotes(inp, rows, columns, i):    
  fig = plt.figure()
  fig = fig.add_subplot(111)
  plt.ylim((100, 270))
  plt.xlim((0, 7000))
  plt.xlabel('time')
  plt.ylabel('pitch')
  for note in inp:
      width = note["duration"]
      height = 2
      x = note["time"]
      y = note["midi"]*(height+1)
      fig.add_patch(patches.Rectangle((x, y), width, height, color='purple' ))
  return fig

def drawAddNotes(inp, fig):    
  for note in inp:
      width = note["duration"]
      height = 2
      x = note["time"]
      y = note["midi"]*(height+1)
      fig.add_patch(patches.Rectangle((x, y), width, height, color='purple'))
  return fig

chordFig = drawChords(chords, threshold, 1,1,1)
drawAddNotes(notes,chordFig)

# HELPERS

In [11]:
def sample(a, temperature=2.0):
  a = np.log(a) / temperature 
  dist = np.exp(a)/np.sum(np.exp(a)) 
  choices = range(len(a)) 
  return np.random.choice(choices, p=dist)


def windowSplitSequenceXY(data, window):
  window_x = []
  window_y = []
  for i in range(len(data)-window):
    seq = data[i: i+window]
    label = data[i+window]
    window_x.append(seq)
    window_y.append(label)
  return window_x, window_y

# LOAD AND PREPROCESS CHORDS

In [12]:
"""
LOAD DATA
"""
chordsData = pd.read_csv(DIRECTORY + CHORDS_DATAFRAME_NAME)
print(len(chordsData), "chords")

# remove single notes
chordsData = chordsData[chordsData['forte-cardinality'] > 1]

"""
FILTER COMMON
"""
counts = chordsData["forte"].value_counts()
forte_index = counts.index.tolist()
threshold = 300 #minimum forte occurences
rare = []
common = []
for i, count in enumerate(counts):
  if count < threshold:
    rare.append(forte_index[i])
  else:
    common.append(forte_index[i])
    
print('Number of unique rare forte:', len(rare))
print('Number of unique common forte:', len(common))

common_df = chordsData[chordsData["forte"].isin(common)]
rare_df = chordsData[chordsData["forte"].isin(rare)]

print('Number of rare forte:', len(rare_df))
print('Number of common forte:', len(common_df))

"""
LABEL ENCODE FORTE
"""
forteEncoder = LabelEncoder()
common_df = common_df.copy()
common_df["forte-label-encoded"] = forteEncoder.fit_transform(common_df["forte"])
print('forte vocab size:', len(common_df["forte"].unique()))

"""
NORMALIZE TIME
"""
# ## Visualise Time
# displayRange = 400
# common_df['duration'].plot.hist(bins=40, range=(0,displayRange))
# common_df['relativeTime'].plot.hist(bins=40, range=(0,displayRange))

TICKS_PER_BEAT = 480
SUBDIVISION = 8
SUBDIV_TICKS = TICKS_PER_BEAT/SUBDIVISION
MAX_DURATION = SUBDIVISION * 2
MAX_RELATIVE_TIME = SUBDIVISION

#ticks to beats
time_norm_df = common_df.copy()
time_norm_df["relativeTime"] = common_df["relativeTime"]/SUBDIV_TICKS
time_norm_df["duration"] = common_df["duration"]/SUBDIV_TICKS

#quantize
time_norm_df["duration"] = time_norm_df["duration"].round().astype(int)
time_norm_df["relativeTime"] = time_norm_df["relativeTime"].round().astype(int)
time_norm_df

#cap
time_norm_df["duration"] = time_norm_df["duration"].clip(1, MAX_DURATION)
time_norm_df["relativeTime"] = time_norm_df["relativeTime"].clip(1, MAX_RELATIVE_TIME)

"""
BIN VELOCITIES
"""
velocity_df = time_norm_df.copy()
# velocity_df['velocity'].plot.hist(bins=10, range=(20, 120))

velocity_bins = [0, 40, 50, 60, 70, 80, 130]
velocity_labels = [40, 45, 55, 65, 75, 85]
velocity_df['velocity'] = pd.cut(velocity_df['velocity'], bins=velocity_bins, labels=velocity_labels).astype(float)

final_df = velocity_df.copy()
final_df[['root', 'velocity', 'relativeTime', 'duration', 'forte']]

79833 chords
Number of unique rare forte: 243
Number of unique common forte: 42
Number of rare forte: 7631
Number of common forte: 66622
forte vocab size: 42


,root,velocity,relativeTime,duration,forte
0,9,55.0,1,3,4-26
1,0,55.0,4,2,4-24
2,0,40.0,2,1,2-4
3,4,85.0,1,1,2-1
4,5,55.0,1,3,4-20
...,...,...,...,...,...
79827,5,40.0,8,12,4-22A
79829,11,55.0,8,16,3-7A
79830,2,45.0,8,12,2-5
79831,4,40.0,1,16,2-5


# CHORD DICT <-> CHORD FEATURE VECTOR

In [17]:
forte_num_classes = len(common)
root_num_classes = 12
relative_time_num_classes = MAX_RELATIVE_TIME + 1
duration_num_classes = MAX_DURATION + 1
velocity_num_classes = len(velocity_labels)

def featureVecToChordDict(hotEncodedFeatureVector):
  root = sample(hotEncodedFeatureVector[:root_num_classes], 0.4)
  forte = sample(hotEncodedFeatureVector[root_num_classes:][:forte_num_classes], 0.4)
  relativeTime = np.argmax(hotEncodedFeatureVector[root_num_classes:][forte_num_classes:][:relative_time_num_classes])
  duration = np.argmax(hotEncodedFeatureVector[root_num_classes:][forte_num_classes:][relative_time_num_classes:][:duration_num_classes])
  velocity = np.argmax(hotEncodedFeatureVector[root_num_classes:][forte_num_classes:][relative_time_num_classes:][duration_num_classes:][:velocity_num_classes])

  return {
      "root": root,
      "forte": forteEncoder.inverse_transform([forte])[0],
      "relativeTime": relativeTime,
      "duration": duration,
      "velocity": velocity_labels[velocity]
  }

def chordDictToFeatureVec(chordDict):
  hot_encoded_root = keras.utils.to_categorical(chordDict['root'], num_classes=root_num_classes)
  
  forte = forteEncoder.transform([chordDict['forte']])[0]
  hot_encoded_forte = keras.utils.to_categorical(forte, num_classes=forte_num_classes)

  hot_encoded_duration = keras.utils.to_categorical(chordDict['duration'], num_classes=duration_num_classes)
  hot_encoded_relative_time = keras.utils.to_categorical(chordDict['relativeTime'], num_classes=relative_time_num_classes)

  velocity = velocity_labels.index(chordDict['velocity'])
  hot_encoded_velocity = keras.utils.to_categorical(velocity, num_classes=velocity_num_classes)

  return np.concatenate([hot_encoded_root, hot_encoded_forte, hot_encoded_relative_time, hot_encoded_duration, hot_encoded_velocity])

print('root_num_classes:', root_num_classes)
print('duration_num_classes:', duration_num_classes)
print('relative_time_num_classes:', relative_time_num_classes)
print('forte_num_classes:', forte_num_classes)
print('velocity_num_classes:', velocity_num_classes)
print('total:', root_num_classes + duration_num_classes + relative_time_num_classes + forte_num_classes + velocity_num_classes)

root_num_classes: 12
duration_num_classes: 17
relative_time_num_classes: 9
forte_num_classes: 42
velocity_num_classes: 6
total: 86


# WINDOW + VECTORIZE FOR MODEL

In [14]:
chord_vectors = []
for i, chord in final_df.iterrows():
  chord_vectors.append(chordDictToFeatureVec(chord))    

window = 16
inputs, labels = windowSplitSequenceXY(chord_vectors, window)
inputs = np.array(inputs)
labels = np.array(labels)
print(inputs.shape)
print(labels.shape)

(66606, 16, 86)
(66606, 86)


# CHORDS MODEL

In [15]:
def buildModel(node_count, dropout):
  model = Sequential()
  model.add(LSTM(node_count, input_shape=(inputs.shape[1], inputs.shape[2]), return_sequences=True))
  model.add(Dropout(dropout))
  model.add(SeqSelfAttention(attention_activation='sigmoid'))
  model.add(Dropout(dropout))
  model.add(LSTM(node_count, input_shape=(inputs.shape[1], inputs.shape[2])))
  model.add(Dropout(dropout))
  model.add(Dense(node_count, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(node_count, activation='relu'))
  model.add(Dropout(dropout))
  model.add(Dense(labels.shape[1], activation='sigmoid')) 

  optimizer = optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  model.build(input_shape=(len(inputs), window, 1))
  return model

# FIT CHORDS

In [ ]:
node_count = 512
dropout = 0.3
epochs = 50
model = buildModel(node_count, dropout)

weight_path = DIRECTORY + CHORDS_WEIGHTS_NAME
print('Weights to:', weight_path)
checkpoint = ModelCheckpoint(
    weight_path,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

History = model.fit(np.array(inputs), np.array(labels), epochs=epochs, batch_size=1024, verbose=1, callbacks=callbacks_list, validation_split=0.2)
plt.plot(History.history["loss"])
plt.plot(History.history["val_loss"])
print('cost:', History.history["loss"][-1])

In [ ]:
plt.plot(History.history["accuracy"])
plt.plot(History.history["val_accuracy"])

# PREDICT

In [21]:
node_count = 512
dropout = 0.3
model = buildModel(node_count, dropout)
weight_path = DIRECTORY + CHORDS_WEIGHTS_NAME
model.load_weights(weight_path)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 16, 512)           1226752   
_________________________________________________________________
dropout_10 (Dropout)         (None, 16, 512)           0         
_________________________________________________________________
seq_self_attention_2 (SeqSel (None, None, 512)         32833     
_________________________________________________________________
dropout_11 (Dropout)         (None, None, 512)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)              

In [22]:
test_inputs = inputs[0]
chord_predictions = []
test_count = 10
for i in range(test_count):
  prediction = model.predict(np.array([test_inputs]))[0]
  chordDict = featureVecToChordDict(prediction)
  test_inputs = np.append(test_inputs[1:], prediction).reshape(window, -1)
  chord_predictions.append(chordDict)

  print(chordDict["root"], chordDict["forte"], chordDict["duration"], chordDict["relativeTime"], chordDict["velocity"])

0 2-6 1 2 55
0 4-26 1 2 55
4 3-11B 1 2 55
9 4-26 1 2 55
5 2-5 1 6 55
11 4-16A 2 6 55
9 3-11B 2 6 55
2 3-11B 2 8 55
4 2-4 2 8 55
5 2-6 2 8 55


# CHORD DICT <-> MIDI

In [23]:
FACTOR = SUBDIV_TICKS * 8
def chordDictToMidi(chordDict, pastDuration=0):
  root = chordDict["root"]
  forte = chordDict["forte"]
  velocity = chordDict["velocity"]
  duration = chordDict["duration"] * FACTOR
  relativeTime = chordDict["relativeTime"]  * FACTOR
  chord = music21.chord.fromForteClass(forte).transpose(root)
  midiNotes = [p.midi for p in chord.pitches]

  midiEvents = []
  for i in range(len(midiNotes)):
    time = max(0, relativeTime-pastDuration) #if the previous note had a duration, the next events will be note_off, which will push the next chord even further back
    if(i != 0):
      time = 0
    note = midiNotes[i]
    message = mido.Message('note_on', note=int(note), velocity=int(velocity), time=int(time))
    midiEvents.append(message)
  
  for i in range(len(midiNotes)):
    time = duration
    if(i != 0):
      time = 0
    note = midiNotes[i]
    message = mido.Message('note_off', note=int(note), velocity=int(velocity), time=int(time))
    midiEvents.append(message)

  return midiEvents


def chordDictSeriesToMidiFile(chordDicts):
  outputMidiFile = MidiFile()
  # bpm = featureVectors[0][idxOfBpmFeature] #sample first vector for bpm
  bpm = 120 #sample first vector for bpm
  test_midiEvents = []
  lastChordDuration = 0
  for chordDict in chordDicts:     
    events = chordDictToMidi(chordDict, lastChordDuration)
    for event in events:
      test_midiEvents.append(event)
    lastChordDuration = chordDict["duration"] * FACTOR

  test_midiEvents.append(mido.MetaMessage('time_signature', numerator=4, denominator=4))
  test_midiEvents.append(mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm)))
  outputTrack = mido.MidiTrack(test_midiEvents)
  outputMidiFile.tracks.append(outputTrack)
  return outputMidiFile

In [ ]:
 now = datetime.now()

current_time = now.strftime("%M")

##### CHORD PREDICT
print('========= PREDICTING CHORDS')
test_inputs = inputs[0]
chord_predictions = []
test_count = 80
for i in range(test_count):
  prediction = model.predict(np.array([test_inputs]))[0]
  chordDict = featureVecToChordDict(prediction)
  test_inputs = np.append(test_inputs[1:], prediction).reshape(window, -1)
  print(chordDict["root"], chordDict["forte"], chordDict["duration"], chordDict["relativeTime"], chordDict["velocity"])
  chord_predictions.append(chordDict)


print(len(chord_predictions))
music_gen_file = chordDictSeriesToMidiFile(chord_predictions)
music_gen_file.save(DIRECTORY + GENERATED_CHORDS_FILE)

# LOAD AND PREPROCESS NOTES

In [29]:
notesData = pd.read_csv(DIRECTORY + NOTES_DATAFRAME_NAME)
print(len(notesData), "notes")
notesData["chordRoot"] = notesData["chordPitch"]

midiEncoder = LabelEncoder()
midi_notes_df = notesData.copy()

common_notes_df = midi_notes_df[midi_notes_df["chordForte"].isin(common)].copy()

notes_time_norm_df = common_notes_df.copy()
notes_time_norm_df["relativeTime"] = common_notes_df["relativeTime"]/SUBDIV_TICKS
notes_time_norm_df["duration"] = common_notes_df["duration"]/SUBDIV_TICKS

#quantize
notes_time_norm_df["duration"] = notes_time_norm_df["duration"].round().astype(int)
notes_time_norm_df["relativeTime"] = notes_time_norm_df["relativeTime"].round().astype(int)
notes_time_norm_df

#cap
notes_time_norm_df["duration"] = notes_time_norm_df["duration"].clip(1, MAX_DURATION)
notes_time_norm_df["relativeTime"] = notes_time_norm_df["relativeTime"].clip(1, MAX_RELATIVE_TIME)

notes_velocity_df = notes_time_norm_df.copy()
notes_velocity_df['velocity'] = pd.cut(notes_velocity_df['velocity'], bins=velocity_bins, labels=velocity_labels).astype(float)

#finalize df
notes_df = notes_velocity_df.copy()
notes_df

83244 notes


,Unnamed: 0,midi,velocity,duration,relativeTime,chordPitch,chordForte,chordQuality,chordRoot
0,0,76,85.0,1,1,9.0,4-26,minor,9.0
1,1,75,75.0,1,1,9.0,4-26,minor,9.0
2,2,72,85.0,1,7,7.0,3-9,other,7.0
3,3,74,85.0,1,1,7.0,3-9,other,7.0
4,4,77,75.0,2,1,4.0,3-8B,other,4.0
...,...,...,...,...,...,...,...,...,...
83239,83239,52,40.0,13,2,11.0,3-7A,minor,11.0
83240,83240,43,45.0,16,8,11.0,3-7A,minor,11.0
83241,83241,64,65.0,6,1,11.0,3-7A,minor,11.0
83242,83242,42,55.0,7,8,0.0,4-20,major,0.0


# NOTE DICT <-> NOTE FEATURE VECTOR

In [26]:
midi_num_classes=128

def featureVecToNoteDict(hotEncodedFeatureVector):
  midi = np.argmax(hotEncodedFeatureVector[:midi_num_classes])
  relativeTime = sample(hotEncodedFeatureVector[midi_num_classes:][:relative_time_num_classes], 0.7)
  duration = sample(hotEncodedFeatureVector[midi_num_classes:][relative_time_num_classes:][:duration_num_classes], 0.7)
  velocity = np.argmax(hotEncodedFeatureVector[midi_num_classes:][relative_time_num_classes:][duration_num_classes:][:velocity_num_classes])
  
  return {
      "midi" : midi, 
      "relativeTime": relativeTime,
      "duration": duration,
      "velocity": velocity_labels[velocity],
      # "chordRoot": chordRoot,
      # "chordForte": forteEncoder.inverse_transform([chordForte])[0]
  }

def noteDictToFeatureVec(noteDict):
  hot_encoded_midi = keras.utils.to_categorical(noteDict['midi'], num_classes=midi_num_classes)
  hot_encoded_root = keras.utils.to_categorical(noteDict['chordRoot'], num_classes=root_num_classes)
  
  forte = forteEncoder.transform([noteDict['chordForte']])[0]
  hot_encoded_forte = keras.utils.to_categorical(forte, num_classes=forte_num_classes)

  hot_encoded_duration = keras.utils.to_categorical(noteDict['duration'], num_classes=duration_num_classes)
  hot_encoded_relative_time = keras.utils.to_categorical(noteDict['relativeTime'], num_classes=relative_time_num_classes)

  
  velocity = velocity_labels.index(noteDict['velocity'])
  hot_encoded_velocity = keras.utils.to_categorical(velocity, num_classes=velocity_num_classes)

  return np.concatenate([hot_encoded_midi, hot_encoded_relative_time, hot_encoded_duration, hot_encoded_velocity, hot_encoded_root, hot_encoded_forte])



print('midi_num_classes:', midi_num_classes)
print('duration_num_classes:', duration_num_classes)
print('relative_time_num_classes:', relative_time_num_classes)
print('forte_num_classes:', forte_num_classes)
print('velocity_num_classes:', velocity_num_classes)

midi_num_classes: 128
duration_num_classes: 17
relative_time_num_classes: 9
forte_num_classes: 42
velocity_num_classes: 6


# WINDOW + VECTORIZE FOR MODEL

In [31]:
note_vectors = []
for i, note in notes_df.iterrows():
  note_vectors.append(noteDictToFeatureVec(note))

window = 16
note_inputs, note_labels = windowSplitSequenceXY(note_vectors, window)
note_inputs = np.array(note_inputs)
note_labels = np.array(note_labels)

# for labels, remove chord
note_labels_final = []
for i in range(len(note_labels)):
  label_vector_length = (midi_num_classes + duration_num_classes + relative_time_num_classes + velocity_num_classes)
  note_labels_final.append(note_labels[i][:label_vector_length])

print('label shape with chords:', note_labels.shape)
note_labels = np.array(note_labels_final)
print('label shape without chords:', note_labels.shape)
print('expected ', midi_num_classes + duration_num_classes + relative_time_num_classes + velocity_num_classes + root_num_classes + forte_num_classes)
note_labels[0]

label shape with chords: (72704, 214)
label shape without chords: (72704, 160)
expected  214


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0.], dtype=float32)

# NOTES MODEL

In [33]:
def buildNotesModel():
  model = Sequential()
  count = 512
  model.add(LSTM(512, input_shape=(note_inputs.shape[1], note_inputs.shape[2]), return_sequences=True))
  model.add(Dropout(0.6))
  model.add(SeqSelfAttention(attention_activation='sigmoid'))
  model.add(LSTM(512, input_shape=(note_inputs.shape[1], note_inputs.shape[2])))
  model.add(Dropout(0.2))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(note_labels.shape[1], activation='sigmoid')) 

  optimizer = optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  model.build(input_shape=(len(note_inputs), window, 1))
  return model

# FIT NOTES

In [ ]:
notes_model = buildNotesModel()

notes_weight_path = DIRECTORY + NOTES_WEIGHTS_NAME
notes_checkpoint = ModelCheckpoint(
    notes_weight_path,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
notes_callbacks_list = [notes_checkpoint]

Notes_Hist = notes_model.fit(np.array(note_inputs), np.array(note_labels), epochs=50, batch_size=1024, verbose=1, callbacks=notes_callbacks_list, validation_split=0.2)
plt.plot(Notes_Hist.history["loss"], color='red')
plt.plot(Notes_Hist.history["val_loss"], color='green')
print('cost:', Notes_Hist.history["loss"][-1])

In [ ]:
plt.plot(Notes_Hist.history['accuracy'])

# PREDICT NOTES

In [38]:
notes_model = buildNotesModel()
notes_weight_path = DIRECTORY + NOTES_WEIGHTS_NAME
notes_model.load_weights(notes_weight_path)
notes_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 16, 512)           1488896   
_________________________________________________________________
dropout_19 (Dropout)         (None, 16, 512)           0         
_________________________________________________________________
seq_self_attention_4 (SeqSel (None, None, 512)         32833     
_________________________________________________________________
lstm_9 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_20 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)              

In [ ]:
current_chord_idx = 0
current_chord = chordDictToFeatureVec(chord_predictions[current_chord_idx])
note_time = chord_predictions[current_chord_idx]['duration'] + chord_predictions[current_chord_idx]['relativeTime']

note_test_inputs = note_inputs[0]
note_predictions = []
note_test_count = 10
for i in range(note_test_count):
  prediction = notes_model.predict(np.array([note_test_inputs]))[0]
  noteDict = featureVecToNoteDict(prediction)  
  nextInput = np.append(prediction, current_chord[:root_num_classes + forte_num_classes])
  note_test_inputs = np.append(note_test_inputs[1:], nextInput).reshape(window, -1)

  currentChordDict = chord_predictions[current_chord_idx]
  print('midi:', noteDict["midi"], 'dur:', noteDict["duration"], 'rtime:', noteDict["relativeTime"], noteDict["velocity"], 'chord:', currentChordDict['root'], 'chord forte:', currentChordDict['forte'])
  note_predictions.append(noteDict)

  note_time = note_time - noteDict["duration"] - noteDict["relativeTime"]
  if note_time < 0:
    current_chord_idx += 1
    if current_chord_idx >= len(chord_predictions):
      break
    current_chord = chordDictToFeatureVec(chord_predictions[current_chord_idx])
    note_time += chord_predictions[current_chord_idx]['duration'] + chord_predictions[current_chord_idx]['relativeTime']    

# NOTE DICT <-> MIDI

In [40]:
NOTE_FACTOR = SUBDIV_TICKS * 8
def noteDictToMidi(noteDict, pastDuration=0):
  midi = noteDict["midi"]
  velocity = noteDict["velocity"]
  duration = noteDict["duration"] * NOTE_FACTOR
  relativeTime = noteDict["relativeTime"]  * NOTE_FACTOR

  midiEvents = []
  time = max(0, relativeTime-pastDuration) #if the previous note had a duration, the next events will be note_off, which will push the next chord even further back
  message = mido.Message('note_on', note=int(midi), velocity=int(velocity), time=int(time))
  midiEvents.append(message)
  message = mido.Message('note_off', note=int(midi), velocity=int(velocity), time=int(duration))
  midiEvents.append(message)
  return midiEvents


def noteDictSeriesToMidiFile(noteDicts):
  outputMidiFile = MidiFile()
  bpm = 120 #sample first vector for bpm
  test_midiEvents = []
  lastNoteDuration = 0
  for noteDict in noteDicts:     
    events = noteDictToMidi(noteDict, lastNoteDuration)
    for event in events:
      test_midiEvents.append(event)
    lastNoteDuration = noteDict["duration"] * NOTE_FACTOR

  test_midiEvents.append(mido.MetaMessage('time_signature', numerator=4, denominator=4))
  test_midiEvents.append(mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm)))
  outputTrack = mido.MidiTrack(test_midiEvents)
  outputMidiFile.tracks.append(outputTrack)
  return outputMidiFile

# COMBINED PREDICT

### Load prediction start seed

In [42]:
start_chords_df = pd.read_csv(DIRECTORY + SEED_SONG_NAME + 'Chords.csv')

# remove single notes
start_chords_df = start_chords_df[start_chords_df['forte-cardinality'] > 1]
start_chords_common_df = start_chords_df[start_chords_df["forte"].isin(common)]
print('start chords common:', len(start_chords_common_df))

start_chords_common_df["forte-label-encoded"] = forteEncoder.transform(start_chords_common_df["forte"])


"""
NORMALIZE TIME
"""

#ticks to beats
sc_time_norm_df = start_chords_common_df.copy()
sc_time_norm_df["relativeTime"] = start_chords_common_df["relativeTime"]/SUBDIV_TICKS
sc_time_norm_df["duration"] = start_chords_common_df["duration"]/SUBDIV_TICKS

#quantize
sc_time_norm_df["duration"] = sc_time_norm_df["duration"].round().astype(int)
sc_time_norm_df["relativeTime"] = sc_time_norm_df["relativeTime"].round().astype(int)
sc_time_norm_df

#cap
sc_time_norm_df["duration"] = sc_time_norm_df["duration"].clip(1, MAX_DURATION)
sc_time_norm_df["relativeTime"] = sc_time_norm_df["relativeTime"].clip(1, MAX_RELATIVE_TIME)

"""
BIN VELOCITIES
"""
sc_velocity_df = sc_time_norm_df.copy()

velocity_bins = [0, 40, 50, 60, 70, 80, 130]
velocity_labels = [40, 45, 55, 65, 75, 85]
sc_velocity_df['velocity'] = pd.cut(sc_velocity_df['velocity'], bins=velocity_bins, labels=velocity_labels).astype(float)

sc_final_df = sc_velocity_df.copy()
sc_final_df

start_chord_vectors = []
for i, chord in sc_final_df.iterrows():
  start_chord_vectors.append(chordDictToFeatureVec(chord))    

window = 16
start_input_chords, _ = windowSplitSequenceXY(start_chord_vectors, window)
len(start_input_chords[0])

start_notes_df = pd.read_csv(DIRECTORY + SEED_SONG_NAME + 'Notes.csv')

start_notes_df["chordRoot"] = start_notes_df["chordPitch"]

start_midi_notes_df = start_notes_df.copy()

start_common_notes_df = start_midi_notes_df[start_midi_notes_df["chordForte"].isin(common)].copy()

start_notes_time_norm_df = start_common_notes_df.copy()
start_notes_time_norm_df["relativeTime"] = start_common_notes_df["relativeTime"]/SUBDIV_TICKS
start_notes_time_norm_df["duration"] = start_common_notes_df["duration"]/SUBDIV_TICKS

#quantize
start_notes_time_norm_df["duration"] = start_notes_time_norm_df["duration"].round().astype(int)
start_notes_time_norm_df["relativeTime"] = start_notes_time_norm_df["relativeTime"].round().astype(int)
start_notes_time_norm_df

#cap
start_notes_time_norm_df["duration"] = start_notes_time_norm_df["duration"].clip(1, MAX_DURATION)
start_notes_time_norm_df["relativeTime"] = start_notes_time_norm_df["relativeTime"].clip(1, MAX_RELATIVE_TIME)

start_notes_velocity_df = start_notes_time_norm_df.copy()
start_notes_velocity_df['velocity'] = pd.cut(start_notes_velocity_df['velocity'], bins=velocity_bins, labels=velocity_labels).astype(float)

#finalize df
start_notes_final_df = start_notes_velocity_df.copy()

start_note_vectors = []
for i, note in start_notes_final_df.iterrows():
  start_note_vectors.append(noteDictToFeatureVec(note))

start_note_inputs, _ = windowSplitSequenceXY(start_note_vectors, window)
start_note_inputs = np.array(start_note_inputs)
start_notes_final_df

start chords common: 163


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,midi,velocity,duration,relativeTime,chordPitch,chordForte,chordQuality,chordRoot
0,0,66,75.0,6,1,4,2-5,other,4
1,1,66,75.0,6,8,4,2-5,other,4
2,2,64,55.0,5,8,4,2-5,other,4
3,3,66,65.0,16,8,4,2-5,other,4
9,9,69,85.0,8,8,4,2-3,minor,4
...,...,...,...,...,...,...,...,...,...
181,181,91,75.0,3,4,3,2-4,other,3
182,182,93,75.0,1,2,3,2-4,other,3
183,183,95,65.0,4,4,3,2-4,other,3
184,184,97,65.0,4,3,3,2-4,other,3


In [ ]:
now = datetime.now()

current_time = now.strftime("%M")

##### CHORD PREDICT
print('========= PREDICTING CHORDS')
test_inputs = start_input_chords[0]
chord_predictions = []
test_count = 200
for i in range(test_count):
  prediction = model.predict(np.array([test_inputs]))[0]
  chordDict = featureVecToChordDict(prediction)
  test_inputs = np.append(test_inputs[1:], prediction).reshape(window, -1)
  print(chordDict["root"], chordDict["forte"], chordDict["duration"], chordDict["relativeTime"], chordDict["velocity"])
  chord_predictions.append(chordDict)


print(len(chord_predictions))
music_gen_file = chordDictSeriesToMidiFile(chord_predictions)
music_gen_file.save(DIRECTORY + GENERATED_CHORDS_FILE)

print('========= PREDICTING NOTES')
##### NOTES PREDICT
current_chord_idx = 0
current_chord_dict = chord_predictions[current_chord_idx]
current_chord = chordDictToFeatureVec(current_chord_dict)
note_time = chord_predictions[current_chord_idx]['duration'] + chord_predictions[current_chord_idx]['relativeTime']

note_test_inputs = start_note_inputs[0]
note_predictions = []
note_test_count = 300
for i in range(note_test_count):
  prediction = notes_model.predict(np.array([note_test_inputs]))[0]
  noteDict = featureVecToNoteDict(prediction)  
  
  nextInput = np.append(prediction, current_chord[:root_num_classes + forte_num_classes])
  note_test_inputs = np.append(note_test_inputs[1:], nextInput).reshape(window, -1)
  print('midi:', noteDict["midi"], 'dur:', noteDict["duration"], 'rtime:', noteDict["relativeTime"], noteDict["velocity"], 'chord:', current_chord_dict['root'], 'chord forte:', current_chord_dict['forte'])
  note_predictions.append(noteDict)

  print(note_time)
  note_time = note_time - noteDict["duration"] - noteDict["relativeTime"]
  shouldBreak=False
  while note_time <= 0:    
    current_chord_idx += 1
    if current_chord_idx >= len(chord_predictions):
      shouldBreak = True
      break
    current_chord_dict = chord_predictions[current_chord_idx]
    print(f'next chord: {current_chord_dict["root"]} {current_chord_dict["forte"]}')
    current_chord = chordDictToFeatureVec(current_chord_dict)
    note_time += (current_chord_dict['duration'] + current_chord_dict['relativeTime'])    
  if shouldBreak:
    break


print(len(note_predictions))
note_music_gen_file = noteDictSeriesToMidiFile(note_predictions)
note_music_gen_file.save(DIRECTORY + GENERATED_NOTES_FILE)